In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def logic_win(x,x_r):
    return (x >= x_r[0]) & (x < x_r[1])

#### Import and clean dataset, select out London

In [2]:
path = 'combined_collisions_v3.csv'

df = pd.read_csv(path,low_memory=False)
df.drop(columns='Unnamed: 0',inplace=True)

In [3]:
#Time column has some nans in it, must drop those entries

def is_str(x):
    Nx = len(x)
    I = np.ones(Nx).astype(bool)
    for ii in range(Nx):
        if type(x[ii]) != str:
            I[ii] = False
    return I

df = df.loc[is_str(df.Time.values),:]
df.aadf_Count_point_id = df.aadf_Count_point_id.values.astype(int)

In [4]:
#make hour, year and month columns
    #hour will be rounded 
def get_hour(T):
    Nt = len(T)
    H = np.zeros(Nt).astype(int)
    for ii in range(Nt):
        s = T[ii].split(':')
        h = int(s[0])
        m = int(s[1])
        if m > 30:
            h += 1
        H[ii] = h % 24
    return H

def get_ymd(D):
    Nd = len(D)
    # |
    ymd = np.zeros((Nd,3)).astype(int)
    for ii in range(Nd):
        s = D[ii].split('-')
        for nn in range(3):
            ymd[ii,nn] = int(s[nn])
    return ymd

def reformat_aadf_year(Year):
    Ny = len(Year)
    y = np.zeros(Ny).astype(int)
    for ii in range(Ny):
        y[ii] = int(Year[ii].split('-')[0])
    return y

df[['y','m','d']] = get_ymd(df.Date.values)
df['h'] = get_hour(df.Time.values)
#df['aadf_Year'] = reformat_aadf_year(df.aadf_Year.values)

##### Below selects out region. You can use Dan's feature to select Greater London, or code below for central London.
- additionally I drop all entries with no traffic counter matches

In [5]:
#central London box
Lat_r = [51.475,51.543]
Lon_r = [-0.179,0.023]

I_lat = (df.Latitude > Lat_r[0]) & (df.Latitude < Lat_r[1])
I_lon = (df.Longitude > Lon_r[0]) & (df.Longitude < Lon_r[1])
I = I_lat & I_lon

#df_lond = df.loc[I,:].copy() #selects central London
df_lond = df.loc[df.in_london,:].copy() #this selects Greater London

In [6]:
#drop all data that don't have traffic matches

I = df_lond.match == True
df_lond = df_lond.loc[I,:]
df_lond.aadf_Year = reformat_aadf_year(df_lond.aadf_Year.values)

#### Make neighborhoods feature

In [7]:
#Make a 3 (Lat, N/S) x 6 (Lon, E/W) grid
Nlat = 3
Nlon = 6
Nneigh = Nlon*Nlat

edges_lat = np.linspace(*Lat_r,Nlat+1)
dlat = np.diff(edges_lat)[0]
bins_lat = edges_lat[:-1]+0.5*dlat
edges_lon = np.linspace(*Lon_r,Nlon+1)
dlon = np.diff(edges_lon)[0]
bins_lon = edges_lon[:-1]+0.5*dlon

coord_neigh = [None]*Nneigh

df_lond['neigh'] = 0 #initialize neighborhood feature
ir = 0 #these are lat bins (rows)
ic = 0 #lon bins
for ii in range(Nneigh):
    Ilat = logic_win(df_lond.Latitude,(edges_lat[ir],edges_lat[ir+1]))
    Ilon = logic_win(df_lond.Longitude,(edges_lon[ic],edges_lon[ic+1]))
    I = Ilat & Ilon
    df_lond.loc[I,'neigh'] = ii
    
    coord_neigh[ii] = np.array([bins_lat[ir],bins_lon[ic]])
    
    ic+=1
    if ic == Nlon:
        ic = 0
        ir+=1

In [10]:
#Make a feature for if there is a bike lane of not ('bikelane'). There is no bike lane if all
#the below classifiers are False

f_laneClass = ['aadf_CLT_CARR', 'aadf_CLT_SEGREG', 'aadf_CLT_STEPP', 'aadf_CLT_PARSEG',
       'aadf_CLT_SHARED', 'aadf_CLT_MANDAT', 'aadf_CLT_ADVIS',
       'aadf_CLT_PRIORI', 'aadf_CLT_CONTRA', 'aadf_CLT_BIDIRE',
       'aadf_CLT_CBYPAS', 'aadf_CLT_BBYPAS', 'aadf_CLT_PARKR',
       'aadf_CLT_WATERR', 'aadf_CLT_PTIME']

Inolane = np.ones(df_lond.shape[0]).astype(bool)
for f in f_laneClass:
    Inolane &= (df_lond[f].values == False)
    
df_lond['bikelane'] = ~Inolane

In [11]:
traf_IDs = np.array(df_lond.aadf_Count_point_id.value_counts().index)
Ninc_IDs = np.array(df_lond.aadf_Count_point_id.value_counts().values)

######################################
Nthresh = 1 #this sets which traffic counters are included, they must have at least this many accidents 
######################################

I = Ninc_IDs >= Nthresh #this will select roads to include in the new DataFrame based on if they had at least this many accidents
I.sum()

IDs_df = traf_IDs[I]
Nid = len(IDs_df)

df_road = pd.DataFrame({'ID':IDs_df})
slight_rd = np.zeros(Nid).astype(int)
ser_rd = np.zeros(Nid).astype(int)
total_rd = np.zeros(Nid).astype(int)
for nn,ii in enumerate(IDs_df):
    I = df_lond.aadf_Count_point_id.values == ii
    total_rd[nn] = I.sum()
    slight_rd[nn] = df_lond.loc[I,'slight'].values.sum()
    ser_rd[nn] = total_rd[nn] - slight_rd[nn]
df_road['total'] = total_rd
df_road['slight'] = slight_rd
df_road['serious'] = ser_rd

safety_mode = ['neigh','Road_name','Road_name2','h','Day_of_Week','m','Speed_limit','Junction_Detail','Junction_Control','Pedestrian_Crossing-Physical_Facilities']
aadf_av = ['distance_to_cp','aadf_Pedal_cycles','aadf_Pedal_cycles','aadf_All_motor_vehicles','aadf_All_HGVs','aadf_LGVs','aadf_Buses_and_coaches']
aadf_mode = ['bikelane','aadf_Road_name','aadf_Road_category','aadf_Road_type','aadf_CLT_CARR', 'aadf_CLT_SEGREG', 'aadf_CLT_STEPP', 'aadf_CLT_PARSEG',
       'aadf_CLT_SHARED', 'aadf_CLT_MANDAT', 'aadf_CLT_ADVIS',
       'aadf_CLT_PRIORI', 'aadf_CLT_CONTRA', 'aadf_CLT_BIDIRE',
       'aadf_CLT_CBYPAS', 'aadf_CLT_BBYPAS', 'aadf_CLT_PARKR',
       'aadf_CLT_WATERR', 'aadf_CLT_PTIME','aadf_Link_length_miles']

for f in safety_mode:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mode().values[0]

for f in aadf_mode:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mode().values[0]
        
for f in aadf_av:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mean()
        
median_d = df_road.distance_to_cp.median()
median_bv = df_road.aadf_Pedal_cycles.median()
df_road['total_dnorm'] = df_road['total'].values*median_d/df_road['distance_to_cp'].values
df_road['total_dnorm_bvnorm'] = df_road.total_dnorm.values*median_bv/df_road.aadf_Pedal_cycles.values
df_road['serious_ratio'] = (df_road['serious'].values)/df_road['total'].values

<ipython-input-11-9f85eff6dc66>:56: RuntimeWarning: divide by zero encountered in divide
  df_road['total_dnorm_bvnorm'] = df_road.total_dnorm.values*median_bv/df_road.aadf_Pedal_cycles.values


In [14]:
#add in aadf year availability metadata 
path_year = '../Data/UK/aadf_year_meta.csv'
df_year = pd.read_csv(path_year)

#merge the metadata into df_road
feat_add = list(df_year.columns)
feat_add.remove('Count_point_id')
for c in feat_add:
    df_road[c] = np.zeros(Nid).astype(int)
    for ii,ID in enumerate(df_road.ID.values):
        I = df_year.Count_point_id == ID
        df_road.loc[ii,c] = df_year.loc[I,c].values

#### Here is the final DataFrame, feel free to save your own version

In [15]:
df_road.sample(20)

,ID,total,slight,serious,neigh,Road_name,Road_name2,h,Day_of_Week,m,...,aadf_All_HGVs,aadf_LGVs,aadf_Buses_and_coaches,total_dnorm,total_dnorm_bvnorm,serious_ratio,N_year,Year_start,Year_end,Max_year_gap
272,36916,37,31,6,8,A300,U,9,3,1,...,630.405405,2508.918919,578.270270,76.444661,4.618642,0.162162,22,2000,2021,1
1089,73165,8,8,0,0,A232,U,15,5,4,...,713.875000,2459.750000,706.375000,6.711194,7.993397,0.000000,22,2000,2021,1
1545,6783,3,2,1,0,A223,U,9,2,7,...,1229.000000,5339.666667,310.000000,1.416090,10.643513,0.333333,22,2000,2021,1
1626,74909,3,2,1,0,A3036,A214,7,1,4,...,676.333333,3466.666667,1034.666667,8.305173,3.055429,0.333333,22,2000,2021,1
1715,73004,2,2,0,0,A1000,U,16,6,1,...,314.000000,1255.000000,228.000000,2.338325,4.006101,0.000000,22,2000,2021,1
483,79993,24,21,3,0,A503,U,9,5,5,...,1579.583333,4578.833333,641.875000,15.673294,9.598627,0.125000,18,2000,2017,1
1054,16401,9,8,1,7,A40,na0,20,4,3,...,200.111111,889.555556,3390.777778,30.945049,1.758870,0.111111,18,2000,2017,1
517,48350,22,20,2,0,A2199,C,9,3,4,...,308.500000,2639.318182,381.500000,9.777779,10.361980,0.090909,22,2000,2021,1
894,48544,11,10,1,0,A3218,A3220,8,4,1,...,329.636364,2457.545455,432.181818,26.464183,3.935691,0.090909,22,2000,2021,1
620,48159,18,15,3,7,A400,A400,15,5,5,...,415.555556,2263.777778,1396.777778,50.160409,6.754624,0.166667,22,2000,2021,1


In [ ]:
##save dataframe

# path = 'df_road_v2.csv'
# df_road.to_csv(path,index_label=False)